# Welcome to the HPE Container Platform API series

Jupyter Notebook can be found here:

**Requirements:**
HPE Container Platform instance
IP address of the HPE Container Platform's controller host

**Utilities:** 
Postman 
Jupyter Notebook server


## The HPE Container Platform API Reference
Before you can call the HPE Container Platform API, you need to know what calls can be placed. The REST API reference documentation describes each object type, along with the supported operations, request input parameters, response model and response codes. 
To access the REST API reference documentation, obtain the IP address or hostname of the current active HPE Container Platform controller host from the administrator of the platform. Then in a web browser, navigate to the following URL:

``` 
https://[Controller-IP-address]/apidocs
```

All the API calls are in the form: 
``` 
An HTTP VERB (GET , POST, DELETE, PUT, PATCH) to target a API object: https://[Controller-IP-address]:8008/api/v2/[object]
```

## Session Authentication
With the exception of some API calls, most of the REST API calls you can do against the HPE Container Platform API requires authentication. HPE Container Platform uses a ‘session location’ to use as operation context. You request an authentication session location by issuing an authentication request in the following form:
* Call the API to request a new login session, providing username/password credentials in the JSON body. It is a valid tenant user/admin or platform administrator credentials) 
* Extract the location from JSON response header
* For each subsequent call, set a new HTTP Header with its key set to *X-BDS-SESSION* and its value set to the session location value 

``` 
Note: the session location will expire after 24 hours. 
```   

If you are not already familiar with REST API calls and Postman, I encourage you to check-out the [Understanding API basics and the value they provide](https://developer.hpe.com/blog/understanding-api-basics-and-the-value-they-provide) blog on HPE Developer Community portal. It explains you REST API concepts such as HTTP verbs you call against the API, the headers, and payloads and how to use Postman to make API calls.  

### Step 1:
You call the HPE Container Platform API to request a new login session location. You provide the credentials of a tenant user, tenant admin or Site admin depending on your role. Let's do it first in Postman and then in cURL command.

**Postman:**
Postman can be used to issue REST API calls to the HPE Container Platform. The following capture shows the information passed by Postman to the API. It is a POST operation against /api/v2/session to create a session object. The header specifies JSON as the exchange format and the body provides the user credentials. The target HPE Container Platform controller IP address and the user credentials are defined in a group of variables called a Postman environment. These variables typically describe a given target environment and are expected to be set before the API calls. Postman also allows you to extract the session location from the response header and store it in a Postman envrionment variable for later use. This is accomplished by using a single line of JavaScript code as shown in the Tests tab. 

<img src="Pictures/HPECP-Postman-post-session.gif" height="800" width="600" align="left">

**cURL:** Let's do the equivalent API call to create the session using cURL. Information on cURL can be found [here](https://curl.haxx.se/)

In [23]:
controller_ip_name="controller.hpedevlab.net:8080"
username="demouser"
password="password"

In [24]:
sessionlocation=$(curl -k -i -s --request POST "https://${controller_ip_name}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'"
}' | grep Location | awk '{print $2}')
echo $sessionlocation

/api/v2/session/5668932d-1e1d-47e2-a58d-c0637575fdc9


### Step 2:
Save the value of the session _Location_ from the response header above, to use it in all your subsequent API calls. For example, here I call the HPE Container Platform API to get the list of application templates available to me.

**Postman:**

<img src="Pictures/HPECP-Postman-Get-template.gif" height="800" width="600" align="left">

**cURL:**

In [25]:
curl -k -s --request GET "https://${controller_ip_name}/api/v2/template" \
--header 'X-BDS-SESSION: /api/v2/session/0fccb893-02b3-47fc-982e-d219d8ae0cf9' \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' | jq

{
  "_links": {
    "self": {
      "href": "/api/v2/template"
    }
  },
  "_embedded": {
    "templates": [
      {
        "_links": {
          "self": {
            "href": "/api/v2/template/3"
          },
          "all_templates": {
            "href": "/api/v2/template"
          }
        },
        "_embedded": {
          "label": {
            "_links": {
              "self": {
                "href": "/api/v2/template/3/label"
              }
            },
            "name": "Centos 7 example",
            "description": "system-created example template"
          },
          "clusterspec": {
            "_links": {
              "self": {
                "href": "/api/v2/template/3/clusterspec"
              }
            },
            "nodegroup": {
              "catalog_entry_distro_id": "bluedata/centos7",
              "role_configs": [
                {
                  "role_id": "controller",
                  "flavor_name": "Large"
                },
     